In [53]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torchsummary import summary
from tqdm import tqdm
import random
import model_architecture

# Data import

In [54]:
col_names = ["Date","Time","Tag Comment","Motor Vol Flow","Motor Flow In","Motor Flow Out","Motor Surf top","Motor Surf Side","Temp Pt100 RTD winding U", \
            "Temp Pt100 RTD winding V", "Temp Pt100 RTD winding W", "Temp PTC thermistor winding", "Temp Feed-through plate", \
            "Lead intersection", "Temp Cable gland", "Temp Terminal box seal", "Temp Terminal", "Temp Ambient", "Frequency", "Voltage", \
            "Current", "Power factor", "Input power", "Output power", "Torque", "Speed", "Slip"]
df = pd.read_excel('./HDE-INT-MOT-20221028-01_HT8_03_N85z-4_OC2_50Hz_70C_R_000759_221107_100743.GEV.xlsx',header=32)
df.columns = col_names
df = df.drop([0],axis=0)
df = df.drop(["Tag Comment","Date","Temp PTC thermistor winding"],axis=1)
df["Time"] = list(np.arange(19206))
df = df[df["Frequency"]!="INVALID"]
print(df.shape)
df.head(5)

(19204, 24)


,Time,Motor Vol Flow,Motor Flow In,Motor Flow Out,Motor Surf top,Motor Surf Side,Temp Pt100 RTD winding U,Temp Pt100 RTD winding V,Temp Pt100 RTD winding W,Temp Feed-through plate,...,Temp Ambient,Frequency,Voltage,Current,Power factor,Input power,Output power,Torque,Speed,Slip
1,0,49.6,41.2,39.3,29.2,29,30.9,30.8,30.7,19.9,...,20.4,50,400.4,100.86,0.35,42.43,26.78,170.98,1495.5,0.3
2,1,49.7,41.2,39.3,29.1,29.2,30.9,30.8,30.8,19.8,...,20.4,50,400.4,100.87,0.35,42.41,26.78,171.02,1495.6,0.28
3,2,49.9,41.2,39.3,29,29,30.9,30.8,30.8,19.9,...,20.4,50,400.4,100.96,0.35,42.48,26.81,171.21,1495.5,0.29
4,3,49.8,41.2,39.3,29.2,29.1,31,30.9,30.8,19.9,...,20.4,50,400.4,100.88,0.35,42.45,26.81,171.17,1495.5,0.3
5,4,49.7,41.2,39.3,29.3,29,31,30.9,30.9,20.2,...,20.4,50,400.3,107.1,0.449,57.71,35.88,229.55,1492.6,0.52


# Data Augmentation

In [63]:
df_p = df
X = df_p.to_numpy(dtype=np.float16)
X = torch.Tensor(X)
Y = torch.stack([torch.ones(19204),torch.zeros(19204),torch.zeros(19204)]).view(19204,3)
X_p = X
X.shape

torch.Size([19204, 24])

In [64]:
# Temp Terminal (an Anschlussklemme steigt über 90 oder IN Power über 280 => gelb
# Inp Power über 300 UND Temp Terminal über 100 => rot
print(max(df["Temp Terminal"]))# Temperatur and Anschlussklemme
print(max(df["Input power"]))
# 13 & 19

85.1
268.69


In [130]:
def augment_data(X):
    count_1 = 0
    count_2 = 0
    count_3 = 0
    new_xs = []
    new_ys = []
    for x in X:
        xs = []
        for i in range(0,len(x)):
            n = x[i]
            if i == 13:
                n = n + random.uniform(0,1)
            elif i == 19:
                n = n + random.uniform(0,1)
            else:
                n = n + random.uniform(0,10)
            xs.append(n)
        # get Y values
        if xs[13] > 82.0 or xs[19] > 271:
            new_ys.append([0,0,1])
            count_1+=1
        elif xs[13] > 76.0 or xs[19] > 260.0:
            new_ys.append([0,1,0])
            count_2+=1
        else:
            new_ys.append([1,0,0])
            count_3+=1
        new_xs.append(xs)
    new_xs = torch.Tensor(np.array(new_xs))
    new_ys = torch.Tensor(np.array(new_ys))
    print(count_1,count_2,count_3)
    return new_xs, new_ys

In [131]:
for i in range(0,2):
    xs, ys = augment_data(X_p)
    X = torch.cat((X,xs),0)
    Y = torch.cat((Y,ys),0)
    print(X.shape)
print(X.shape)
print(Y.shape)

6722 5425 7057
torch.Size([844976, 24])
6725 5399 7080
torch.Size([864180, 24])
torch.Size([864180, 24])
torch.Size([864180, 3])


# Ensemble of Neural Network + CNN + LSTM Training

Idee: Ensemble Learner, der das Beste von DNN, CNN, LSTM und Transformer Modellen verbindet

In [133]:
EPOCHS = 1
INIT_LR = 0.0001
device =  torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = model_architecture.Ensemble().to(device)
model= torch.nn.DataParallel(model,device_ids=[0])

lossCE = CrossEntropyLoss()

opt = Adam(model.parameters(), lr=INIT_LR)

H = {"train_loss":[]}

for epoch in range(0, EPOCHS):
    print("Epoch:",str(epoch+1)+"/"+str(EPOCHS))
    # Train Mode
    model.train()
    
    # Initialize
    totalTrainLoss = 0
    totalValLoss = 0
    trainCorrect = 0
    valCorrect = 0

    for i in tqdm(range(0,10000)): # Iterate over Training Examples
        x = X[i].unsqueeze(0)
        y = Y[i].unsqueeze(0)
        x_pred=model(x)
        loss = lossCE(x_pred,y)
        # zero out the gradients, perform the backpropagation step, and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        H["train_loss"].append(float(loss))
    print("Training Loss in Epoch",str(epoch+1),":",np.sum(np.array(H["train_loss"])/(len(X)*(epoch+1))))
       
PATH = "./modelfinal"
torch.save(model.state_dict(), PATH + ".pt")
print(totalTrainLoss)

Epoch: 1/1


100%|██████████| 10000/10000 [00:18<00:00, 532.84it/s]

Training Loss in Epoch 1 : 0.024424462618147257
0
